In [1]:
# load packages
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm 
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import mean_squared_error

import os
import torch
import torch.nn.functional as F
from torch.utils import data
from torchinfo import summary
import torch.nn as nn
import torch.optim as optim
import warnings
import pickle
warnings.filterwarnings('ignore')
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [27]:
from model_classification_best_parameter import model_classification_best_parameters
from model_classification_optuna import model_optuna_classification
from test_classification_0 import test_classification_0
from data_processor import data_processor

# data_processing（等讨论好具体怎么做，以什么为输入数据再写）

In [107]:
model_type = 'MLP'
future_time = 0.25
T = 100
rolling_ratio = 0.5
train_ratio = 0.75
data_type = 'lob'
batch_size = 256
normalization_type = 'minmax'
num_classes = 3
epochs = 100

In [8]:
data_input_processor = data_processor(model_type = model_type, future_time = future_time,T = T,rolling_ratio = rolling_ratio, \
                                      train_ratio = train_ratio,data_type = data_type,batch_size = batch_size,normalization_type = normalization_type)
dataset_train,dataset_val,dataset_test,train_loader,val_loader,test_loader = data_input_processor.getting_data_input()

In [5]:
##比如我们的输入是最近100行的40个feature
##对于LSTM、stacked_LSTM、LSTM-MLP，loader里面输入是[样本量,100,40]形式
##对于MLP，loader里面的输入是[样本量,4000]形式
##对于CNN-LSTM，loader里面的输入是[样本量，1，100，40]形式

##输出数据形式都是[样本量,num_classes]

# model using optuna to get best parameters

In [1]:
model_o = model_optuna_classification(model_type = model_type, epochs = epochs, feature_name = data_type, future_time = future_time,data_train_loader = train_loader,data_val_loader = val_loader,num_classes = num_classes)
para_best = model_o.getting_best_para()

# model_regression using best parameter

In [1]:
##params1改为各个神经网络调好的参数
if model_type == 'CNN-LSTM':
    params1 = {
        'conv_filter_num':32,
        'inception_num':64,
        'LSTM_num':64,
        'leaky_relu_alpha':0.01,
        'num_classes':3,
          }
elif model_type == 'MLP':
    params1 = {
            'inputsize':4000,
            'layer1':400,
            'layer2':50,
            'layer3':10,
         'num_classes':3,
                }
elif model_type == 'LSTM':
    params1 = {
            'num_layers':2,
            'input_size':40,
            'hidden_size':100, 
         'num_classes':3,
                }
elif model_type == 'LSTM-MLP':
    params1 = {
            'num_layers':2,
            'input_size':40,
            'hidden_size':80,  
            'fc_size':20,
         'num_classes':3,
                } 
elif model_type == 'stacked_LSTM':
    params1 = {
            'num_layers_s': 1,
            'input_size_s': 40,
            'hidden_size_s_1': 80,
            'hidden_size_s_2': 150,
            'hidden_size_s_3': 200,
         'num_classes':3,
                }
params2 = {
    'lr':0.00005,
       }
model1 = model_classification_best_parameters(model_type = model_type, epochs = epochs, feature_name = data_type, future_time = future_time,data_train_loader = train_loader,data_val_loader = val_loader,para1 = params1,para2 = params2)
x,xx = model1.train_model()

In [2]:
plt.figure(figsize=(15,6))
plt.plot(x[:], label='train loss')
plt.plot(xx[:], label='validation loss')
plt.legend()

# test the model

In [2]:
model_test = test_classification_0(model_type = model_type, feature_name = data_type, future_time = future_time,data_train_loader = train_loader,data_val_loader = val_loader,data_test_loader = test_loader)
test_result = model_test.test_train()

In [3]:
model_test = test_classification_0(model_type = model_type, feature_name = data_type, future_time = future_time,data_train_loader = train_loader,data_val_loader = val_loader,data_test_loader = test_loader)
test_result = model_test.test_val()

In [4]:
model_test = test_classification_0(model_type = model_type, feature_name = data_type, future_time = future_time,data_train_loader = train_loader,data_val_loader = val_loader,data_test_loader = test_loader)
test_result = model_test.test_test()